In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [3]:
%matplotlib inline

## Load and process country data

In [4]:
countries = pd.read_csv('countries.csv')
countries.head()

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06


In [5]:
def one_hot(df, cols, prefixes=None):
    for col in cols:
        dummies = pd.get_dummies(df[col])
        if prefixes:
            dummies = dummies.rename(columns={x:prefixes(col, x) for x in list(dummies)})
        df = pd.concat((df, dummies), axis=1)
        df = df.drop(col, axis=1)
    return df

countries = one_hot(countries, ["destination_language "], lambda col, x: "dest_%s"%(x))

### add in country demographics

In [6]:
# demographics from 2015 bucketted by age and gender
demographics = pd.read_csv('age_gender_bkts.csv')
demographics = demographics.groupby(["country_destination", "age_bucket", "gender"]).sum()
demographics = demographics.drop(["year"], axis=1)
demographics = demographics.unstack(level=[1, 2])
demographics

population_in_thousands                                    \
age_bucket                              0-4             10-14            100+   
gender                               female     male   female     male female   
country_destination                                                             
AU                                    781.0    824.0    707.0    743.0    4.0   
CA                                    991.0   1045.0    929.0    983.0    7.0   
DE                                   1713.0   1811.0   1800.0   1892.0   14.0   
ES                                   1198.0   1272.0   1124.0   1189.0    9.0   
FR                                   1938.0   2035.0   1894.0   1985.0   19.0   
GB                                   1888.0   1981.0   1690.0   1771.0   13.0   
IT                                   1383.0   1468.0   1388.0   1468.0   15.0   
NL                                    438.0    462.0    493.0    517.0    2.0   
PT                                    225.0    239.0    269.0    285.0    1.0   
US                                  10306.0  10788.0  10346.0  10771.0   61.0   

                                                               ...            \
age_bucket                   15-19             20-24           ...     75-79   
gender               male   female     male   female     male  ...    female   
country_destination                                            ...             
AU                    1.0    737.0    768.0    787.0    820.0  ...     338.0   
CA                    1.0   1009.0   1054.0   1159.0   1203.0  ...     530.0   
DE                    3.0   1974.0   2076.0   2161.0   2266.0  ...    2421.0   
ES                    3.0   1027.0   1087.0   1106.0   1175.0  ...     906.0   
FR                    3.0   1916.0   2016.0   1947.0   2040.0  ...    1223.0   
GB                    3.0   1783.0   1864.0   1957.0   2061.0  ...    1166.0   
IT                    3.0   1411.0   1493.0   1514.0   1601.0  ...    1534.0   
NL                    0.0    484.0    510.0    504.0    527.0  ...     295.0   
PT                    0.0    266.0    277.0    275.0    285.0  ...     244.0   
US                   13.0  10570.0  11025.0  11094.0  11601.0  ...    4532.0   

                                                                            \
age_bucket                    80-84           85-89           90-94          
gender                 male  female    male  female    male  female   male   
country_destination                                                          
AU                    298.0   252.0   199.0   179.0   118.0    94.0   47.0   
CA                    446.0   422.0   318.0   300.0   183.0   153.0   68.0   
DE                   1932.0  1486.0  1016.0   988.0   517.0   491.0  154.0   
ES                    688.0   843.0   559.0   563.0   306.0   256.0  112.0   
FR                    918.0  1137.0   712.0   838.0   405.0   465.0  164.0   
GB                    978.0   896.0   661.0   602.0   365.0   310.0  145.0   
IT                   1191.0  1231.0   808.0   855.0   440.0   436.0  164.0   
NL                    246.0   231.0   158.0   151.0    78.0    69.0   25.0   
PT                    177.0   194.0   122.0   115.0    59.0    45.0   19.0   
US                   3641.0  3394.0  2442.0  2459.0  1441.0  1193.0  541.0   

                                   
age_bucket           95-99         
gender              female   male  
country_destination                
AU                    25.0    9.0  
CA                    42.0   13.0  
DE                    71.0   17.0  
ES                    64.0   22.0  
FR                    82.0   20.0  
GB                    81.0   29.0  
IT                    79.0   22.0  
NL                    15.0    4.0  
PT                     9.0    3.0  
US                   361.0  115.0  

[10 rows x 42 columns]

In [7]:
# join demographics to countries
countries = countries.join(demographics, on="country_destination").fillna(0)

/home/dzhao/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 3 on the right)
  warnings.warn(msg, UserWarning)


## Load and process session data

In [8]:
sessions = pd.read_csv('sessions.csv')
sessions = sessions[(sessions["action_type"] != "-unknown-") & (sessions["device_type"] != "-unknown-")]
sessions = one_hot(sessions, ["action_type", "device_type"])
sessions = sessions.drop(["action", "action_detail"], axis=1)
sessions.head()

,user_id,secs_elapsed,booking_request,booking_response,click,data,message_post,modify,partner_callback,submit,...,Chromebook,Linux Desktop,Mac Desktop,Opera Phone,Tablet,Windows Desktop,Windows Phone,iPad Tablet,iPhone,iPodtouch
0,d1mm9tcy42,319.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,d1mm9tcy42,67753.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,d1mm9tcy42,301.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,d1mm9tcy42,22141.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,d1mm9tcy42,435.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [9]:
user_sessions = sessions.groupby(sessions["user_id"])
user_sessions = user_sessions.sum().fillna(0)
user_sessions.head()

,secs_elapsed,booking_request,booking_response,click,data,message_post,modify,partner_callback,submit,view,...,Chromebook,Linux Desktop,Mac Desktop,Opera Phone,Tablet,Windows Desktop,Windows Phone,iPad Tablet,iPhone,iPodtouch
user_id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,867896.0,1.0,0.0,4.0,9.0,1.0,0.0,1.0,0.0,21.0,...,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
0010k6l0om,463510.0,0.0,0.0,16.0,9.0,0.0,0.0,1.0,0.0,17.0,...,0.0,0.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001wyh0pz8,260784.0,0.0,0.0,66.0,2.0,0.0,0.0,0.0,3.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0028jgx1x1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002qnbzfs5,5714193.0,1.0,0.0,138.0,138.0,16.0,0.0,0.0,15.0,215.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,0.0


## Load and process user data

In [149]:
users = pd.read_csv('train_users_2.csv')
users = users[(users["country_destination"] != "NDF") & (users["country_destination"] != "other")]
users.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
5,osr2jwljor,2010-01-01,20100101215619,2010-01-02,-unknown-,NaN,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US
6,lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
7,0d01nltbrs,2010-01-03,20100103191905,2010-01-13,FEMALE,47.0,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,US


In [150]:
# drop rows with infrequent values
print(users.shape)
threshold = 100
for col in ["signup_method", "signup_flow", "language", "affiliate_channel", "affiliate_provider", "first_affiliate_tracked", "signup_app", "first_device_type", "first_browser"]:
    counts = users[col].value_counts()
    users = users[users[col].isin(counts.index[counts > threshold])]
print(users.shape)

(78814, 16)
(75973, 16)


In [151]:
# one-hot encoding time!
users = one_hot(users, ["language", "signup_method", "signup_flow", "affiliate_channel", "affiliate_provider", "first_affiliate_tracked", "signup_app", "first_device_type", "first_browser"], lambda col, x: "%s_%s"%(col, x))

In [152]:
# join on sessions
users = users.join(user_sessions, on="id").fillna(0)
users.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,country_destination,language_de,language_en,language_es,...,Chromebook,Linux Desktop,Mac Desktop,Opera Phone,Tablet,Windows Desktop,Windows Phone,iPad Tablet,iPhone,iPodtouch
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,US,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,US,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,osr2jwljor,2010-01-01,20100101215619,2010-01-02,-unknown-,0.0,US,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46.0,US,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0d01nltbrs,2010-01-03,20100103191905,2010-01-13,FEMALE,47.0,US,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# join on destination
missing_data = users[["age", "gender", "country_destination"]]
# missing_data = users.merge(countries, on="country_destination", suffixes=("", ""))
missing_data = missing_data.join(demographics, on="country_destination")

# save data with missing values to csv for julia code to import
missing_data = one_hot(missing_data, ["gender"])
missing_data = missing_data.drop(["-unknown-", "OTHER"], axis=1, errors="ignore")

missing_data.drop(["country_destination"], axis=1).to_csv("missing_data.csv", sep=",", index=False)
users_matrix = missing_data.as_matrix()
missing_data.head()

/home/dzhao/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 3 on the right)
  warnings.warn(msg, UserWarning)


,age,country_destination,"(population_in_thousands, 0-4, female)","(population_in_thousands, 0-4, male)","(population_in_thousands, 10-14, female)","(population_in_thousands, 10-14, male)","(population_in_thousands, 100+, female)","(population_in_thousands, 100+, male)","(population_in_thousands, 15-19, female)","(population_in_thousands, 15-19, male)",...,"(population_in_thousands, 80-84, female)","(population_in_thousands, 80-84, male)","(population_in_thousands, 85-89, female)","(population_in_thousands, 85-89, male)","(population_in_thousands, 90-94, female)","(population_in_thousands, 90-94, male)","(population_in_thousands, 95-99, female)","(population_in_thousands, 95-99, male)",FEMALE,MALE
2,56.0,US,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,...,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0
4,41.0,US,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,...,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,0,0
5,0.0,US,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,...,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,0,0
6,46.0,US,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,...,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0
7,47.0,US,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,...,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0


In [ ]:
# generate observations
obs = np.zeros((users_matrix.shape[0] * users_matrix.shape[1], 2), dtype=int)
idx = 0
for i in range(users_matrix.shape[0]):
    if users_matrix[i, 0] > 15 and users_matrix[i, 0] < 105:
        obs[idx] = [i, 0]
        idx += 1
    for j in range(1, users_matrix.shape[1] - 2):
        obs[idx] = [i, j]
        idx += 1
    if users_matrix[i, -2] or users_matrix[i, -1]:
        obs[idx] = [i, users_matrix.shape[1] - 2]
        obs[idx + 1] = [i, users_matrix.shape[1] - 1]
        idx += 2
obs = obs[:idx, :]
np.savetxt("obs.csv", obs, delimiter=",", fmt="%d")

In [ ]:
# if GLRMs aren't doing too hot, going to train regression models instead.
missing_data = missing_data[(missing_data["age"] > 15) & (missing_data["age"] < 105) & ((missing_data["MALE"] == 1) | (missing_data["FEMALE"] == 1))]

# store these to decode missing data results
# norm_data = missing_data[(missing_data["age"] > 15) & (missing_data["age"] < 105)]
# md_range = norm_data.max() - norm_data.min()
# md_mean = norm_data.mean()
# missing_data = ((missing_data - md_mean) / md_range).fillna(0)

# sample uniformly US vs non-US
num_non_us = len(missing_data[missing_data["country_destination"] != "US"])
missing_data[missing_data["country_destination"] == "US"] = missing_data[missing_data["country_destination"] == "US"].sample(num_non_us)
missing_data[missing_data["country_destination"] != "US"] = missing_data[missing_data["country_destination"] != "US"].sample(num_non_us)
missing_data = missing_data.dropna()
missing_data = missing_data.drop(["country_destination"], axis=1)

missing_data.to_csv("clean_missing_data.csv", sep=",", index=False)
missing_data.head()

## At this point, switch over to Julia so we can use the low rank models library to predict the missing ages and genders

In [ ]:
# fill in ages
fill_ages = True
if fill_ages:
    preds = pd.read_csv('glrm_results.csv', header=None)
    users = users.reset_index()
    clean_users = users["age"]
    clean_users[(clean_users <= 15) | (clean_users >= 105)] = None
    users["age"] = users["age"].fillna(preds[0])
else:
    users = users[(users["age"] > 15) & (users["age"] < 105)]
users.head()

In [ ]:
users = one_hot(users, ["gender"])
users = users.drop(["-unknown-", "OTHER", "timestamp_first_active", "id", "first_browser_-unknown-"], axis=1)

In [ ]:
# break apart dates
for col in ["date_account_created", "date_first_booking"]:
    users[col] = pd.to_datetime(users[col])
    df = pd.concat((users[col].dt.year, users[col].dt.month, users[col].dt.day), axis=1)
    df.columns = ["%s_year"%col, "%s_month"%col, "%s_day"%col]
    users = pd.concat((users, df), axis=1)
is_weekend = pd.to_datetime(users["date_first_booking"]).dt.weekday >= 5
users = users.drop(["date_account_created", "date_first_booking"], axis=1)
users = pd.concat((users, is_weekend), axis=1)
users = users.rename(columns={"date_first_booking": "is_weekend"})
users.head()

In [ ]:
users.to_csv("final_dataset.csv", sep=",", index=False)

In [ ]:
users.shape

In [ ]:
users.groupby(["country_destination"]).mean()